In [1]:
%matplotlib notebook
import urllib.request
import gzip
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import tensorflow as tf
from pandas.plotting import scatter_matrix
from sklearn.impute import SimpleImputer
from sklearn.model_selection import StratifiedShuffleSplit
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import confusion_matrix
from tensorflow import keras
from numpy.random import seed
from tensorflow import set_random_seed

# set numpy seed for reproducibility
seed(1)
# set tf seed for reproducibility
set_random_seed(2)
# Turn off GPU usage for tf
os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

In [2]:
def load_covtype_dataset():
    '''Downloads the Cover Type dataset from UCI repository, returning a file handle'''
    CURRENT_DIR = os.getcwd()
    COVTYPE_FILENAME = 'covtype.data'
    COVTYPE_DATA_PATH = os.path.join(CURRENT_DIR, COVTYPE_FILENAME)
    COVTYPE_URL = 'https://archive.ics.uci.edu/ml/machine-learning-databases/covtype/covtype.data.gz'
    if os.path.isfile(COVTYPE_DATA_PATH):
        print('Using local cached copy in', COVTYPE_DATA_PATH)
    else:
        print('Dataset not found locally. Downloading in', COVTYPE_DATA_PATH)
        with urllib.request.urlopen(COVTYPE_URL) as response:
            with gzip.GzipFile(fileobj=response) as uncompressed, open(COVTYPE_DATA_PATH, 'wb') as out_file:
                file_header = uncompressed.read()
                out_file.write(file_header)
    return COVTYPE_DATA_PATH

In [3]:
covtype_file = load_covtype_dataset()

Using local cached copy in /tf/notebook/covtype.data


In [4]:
df_covtype = pd.read_csv(covtype_file, header=None)

In [5]:
df_covtype.describe()

,0,1,2,3,4,5,6,7,8,9,...,45,46,47,48,49,50,51,52,53,54
count,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,...,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000,581012.000000
mean,2959.365301,155.656807,14.103704,269.428217,46.418855,2350.146611,212.146049,223.318716,142.528263,1980.291226,...,0.090392,0.077716,0.002773,0.003255,0.000205,0.000513,0.026803,0.023762,0.015060,2.051471
std,279.984734,111.913721,7.488242,212.549356,58.295232,1559.254870,26.769889,19.768697,38.274529,1324.195210,...,0.286743,0.267725,0.052584,0.056957,0.014310,0.022641,0.161508,0.152307,0.121791,1.396504
min,1859.000000,0.000000,0.000000,0.000000,-173.000000,0.000000,0.000000,0.000000,0.000000,0.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
25%,2809.000000,58.000000,9.000000,108.000000,7.000000,1106.000000,198.000000,213.000000,119.000000,1024.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000
50%,2996.000000,127.000000,13.000000,218.000000,30.000000,1997.000000,218.000000,226.000000,143.000000,1710.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
75%,3163.000000,260.000000,18.000000,384.000000,69.000000,3328.000000,231.000000,237.000000,168.000000,2550.000000,...,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,2.000000
max,3858.000000,360.000000,66.000000,1397.000000,601.000000,7117.000000,254.000000,254.000000,254.000000,7173.000000,...,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,7.000000


In [6]:
def features_renaming(df_covtype):
    '''Rename each column to meaningful labels'''
    # First step: rename the first 14 columns
    first_fourteen_old_feature_names = df_covtype.columns[np.arange(0,14)]
    first_fourteen_new_feature_names = ['Elevation', 'Aspect', 'Slope',
                                        'Horizontal_Distance_To_Hydrology',
                                        'Vertical_Distance_To_Hydrology',
                                        'Horizontal_Distance_To_Roadways',
                                        'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
                                        'Horizontal_Distance_To_Fire_Points',
                                        'Wilderness_Area_1', 'Wilderness_Area_2',
                                        'Wilderness_Area_3', 'Wilderness_Area_4']
    old_to_new_name_mapping = dict(zip(first_fourteen_old_feature_names, first_fourteen_new_feature_names))
    df_covtype.rename(columns=old_to_new_name_mapping, inplace=True)
    # Second step: rename the 40 soil type columns
    soil_type_old_feature_names = df_covtype.columns[np.arange(14,54)]
    soil_type_new_feature_names = ['Soil_Type_' + str(i) for i in np.arange(1,41)]
    old_to_new_name_mapping = dict(zip(soil_type_old_feature_names, soil_type_new_feature_names))
    df_covtype.rename(columns=old_to_new_name_mapping, inplace=True)
    # Last step: rename the last feature (cover type)
    df_covtype.rename(columns={54: 'Cover_Type'}, inplace=True)
    return df_covtype

In [7]:
# Features names are numeric, let's rename each one of them
df_covtype = features_renaming(df_covtype)

In [8]:
# Check the result
df_covtype.head()

,Elevation,Aspect,Slope,Horizontal_Distance_To_Hydrology,Vertical_Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,...,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
0,2596,51,3,258,0,510,221,232,148,6279,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212,-6,390,220,235,151,6225,...,0,0,0,0,0,0,0,0,0,5
2,2804,139,9,268,65,3180,234,238,135,6121,...,0,0,0,0,0,0,0,0,0,2
3,2785,155,18,242,118,3090,238,238,122,6211,...,0,0,0,0,0,0,0,0,0,2
4,2595,45,2,153,-1,391,220,234,150,6172,...,0,0,0,0,0,0,0,0,0,5


In [9]:
# Downcast all features to reduce the overall dataframe dimension
# This step is not needed but it keeps the RAM usage low

# Get the columns header onto a list
df_covtype_headers = df_covtype.columns.values.tolist()
# A list containing the new dtype for each column of interest
features_new_dtype_list = list()
# Iterate through the headers list to assign the new dtype
# for each column
for i in range(len(df_covtype_headers)):
    if i == 3 or i == 4:
        # Horizontal_Distance_To_Hydrology and Vertical_Distance_To_Hydrology
        # could fit in float16
        # but float32 works better with mean and std calculations
        features_new_dtype_list.append('float32')
    elif i < 10:
        # First ten features can have int16
        features_new_dtype_list.append('int16')
    else:
        # the remaining ones are binary integers, so uint8
        # is enough
        features_new_dtype_list.append('uint8')
# A dictionary whose keys are the dataframe columns and values
# are the new dtype ( namely, {'Elevation' : 'int16', ...} )
features_new_dtype_mapping = dict(zip(df_covtype_headers,
                                      features_new_dtype_list))
# Perform the downcasting using the dictionary just created
df_covtype = df_covtype.astype(dtype=features_new_dtype_mapping)

In [10]:
# Check the result
df_covtype.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 581012 entries, 0 to 581011
Data columns (total 55 columns):
Elevation                             581012 non-null int16
Aspect                                581012 non-null int16
Slope                                 581012 non-null int16
Horizontal_Distance_To_Hydrology      581012 non-null float32
Vertical_Distance_To_Hydrology        581012 non-null float32
Horizontal_Distance_To_Roadways       581012 non-null int16
Hillshade_9am                         581012 non-null int16
Hillshade_Noon                        581012 non-null int16
Hillshade_3pm                         581012 non-null int16
Horizontal_Distance_To_Fire_Points    581012 non-null int16
Wilderness_Area_1                     581012 non-null uint8
Wilderness_Area_2                     581012 non-null uint8
Wilderness_Area_3                     581012 non-null uint8
Wilderness_Area_4                     581012 non-null uint8
Soil_Type_1                           581012 

In [11]:
# Check the number of samples for each class value
df_covtype.Cover_Type.value_counts()

2    283301
1    211840
3     35754
7     20510
6     17367
5      9493
4      2747
Name: Cover_Type, dtype: int64

In [12]:
# Let's copy the first 15120 rows onto a new df and
# perform some Exploratory Data Analysis (EDA)
train = df_covtype[:15120].copy()

In [13]:
# Check if stratified sampling has already been done for the training set (it has)
train.Cover_Type.value_counts()

7    2160
6    2160
5    2160
4    2160
3    2160
2    2160
1    2160
Name: Cover_Type, dtype: int64

In [14]:
def elevation_multiple_covtype_distplot(elevations_df, covtype_mapping):
    '''Plot an Elevation's histogram for each Cover Type on the same figure'''
    # Color palette for distplot
    custom_palette = ['#F97EDB', '#49C0EA', '#B49EFC', '#CDAB40',
                      '#F69089', '#76BF3F', '#4CC9A6']
    # Set distplot background
    sns.set_style('darkgrid')
    # Custom bins range for evenly spaced hists
    bins = range(1800,4000,60)
    # Iterate through the dictionary to plot a histogram for each cover type
    for covtype_id, covtype_name in covtype_mapping.items():
        # covtype_id goes from 1 to 7
        # thus subtract 1 for indexing custom_palette
        palette_idx = covtype_id - 1        
        # Create a group for each Cover_Type and return a df satisfying the condition
        # on Cover_Type column
        by_one_covtype = elevations_df.groupby('Cover_Type') \
                                      .apply(lambda x: x[ x['Cover_Type'] == covtype_id ])
        # Plot one Elevation histogram for one group
        ax = sns.distplot(by_one_covtype.Elevation,
                          bins=bins,
                          color=custom_palette[palette_idx], label=covtype_name,
                          hist_kws=dict(alpha=0.8, edgecolor="none"),
                          kde=False)

    # Legend position to upper right
    plt.legend(loc='right', bbox_to_anchor=(1.2, 0.8), ncol=1)
    # Apply proper labeling to the axes
    ax.set(xlabel='Elevation (meters)', ylabel='Count')
    # Avoid cutting off the legend from the figure
    plt.tight_layout()
    # Show the figure (can be omitted in Jupyter Notebooks)
    plt.show()

In [15]:
# Plot an Elevation's histogram for each Cover Type
# to check for possible class separation

# Slice by two columns: Elevation and Cover_Type
elevations = train.loc[:, ['Elevation', 'Cover_Type']]
# Dictionary for mapping each integer target label to its string value
covtype_label_name_dict = {1: 'Spruce/Fir',
                           2: 'Lodgepole Pine',
                           3: 'Ponderosa Pine',
                           4: 'Cottonwood/Willow',
                           5: 'Aspen',
                           6: 'Douglas-fir',
                           7: 'Krummholz'}
# It is clear that classes
# 4 (Willow), 5 (Aspen) and 7 (Krummholz) are easily separable
elevation_multiple_covtype_distplot(elevations, covtype_label_name_dict)

<IPython.core.display.Javascript object>

In [16]:
# Don't need elevations anymore
print('Dereferencing elevations')
del elevations

Dereferencing elevations


In [17]:
# Since the Elevation looks like a promising feature for Cover Type prediction
# let's check the correlation matrix to see which continuos feature
# depends on the Elevation
# Note that we could have used Chi-Squared Test to check for features
# dependency on Cover Type but it is out of scope for this project

# The features we are interested in for the correlation matrix
features_to_check = ['Elevation', 'Aspect',
                     'Slope',
                     'Horizontal_Distance_To_Hydrology',
                     'Vertical_Distance_To_Hydrology',
                     'Horizontal_Distance_To_Roadways',
                     'Hillshade_9am', 'Hillshade_Noon',
                     'Hillshade_3pm',
                     'Horizontal_Distance_To_Fire_Points']
# Same features as above, just with some shorter names
# for visualization purposes
labels_to_plot = ['Elevation', 'Aspect',
                  'Slope', 'HD_Hydro',
                  'VD_Hydro', 'HD_Road',
                  'HS_9am', 'HS_Noon',
                  'HS_3pm', 'HD_Fire']
# Create a dictionary for mapping the labels like this
# {long_label: short_label}
shorter_labels = dict(zip(features_to_check, labels_to_plot))
# Make a copy of the training set because we want to rename
# some columns to fit them on the graph
corr_train = train[features_to_check].copy()
# Do the renaming feeding the mapping dictionary we created
corr_train.rename(columns=shorter_labels, inplace=True)
# Grab the AxesSubplots handle to modify labels padding and rotation
axes = scatter_matrix(corr_train, figsize=(12, 8))
n = len(corr_train.columns)
for x in range(n):
    for y in range(n):
        # for all the axes on the graph..
        ax = axes[x,y]
        # rotate the y-axis labels by 0 (horizontally)..
        ax.yaxis.label.set_rotation(0)
        # add some padding between the labels and their subgraph..
        ax.xaxis.labelpad = 0
        ax.yaxis.labelpad = 20
        # and hide axes value ranges
        ax.set_yticklabels([])
        ax.set_xticklabels([])

<IPython.core.display.Javascript object>

In [18]:
# Let's write down the most correlated features:
# - 'HD_Hydro' and 'VD_Hydro'
# - 'HS_Noon' and 'HS_3pm'
# Plot the same scatter graph zooming on these
corr_train.plot(kind='scatter', x='HD_Hydro', y='VD_Hydro')

<IPython.core.display.Javascript object>

In [19]:
# Looks like there are a lot of HS_3pm equal to zero.
# It could help to impute those with the median
corr_train.plot(kind='scatter', x='HS_Noon', y='HS_3pm')

<IPython.core.display.Javascript object>

In [20]:
# Don't need corr_train df anymore
# Hopefully the garbage collector will clean it up
print('Dereferencing corr_train')
del corr_train

Dereferencing corr_train


In [21]:
# Count Hillshade_3pm zeros on the training set
(train.Hillshade_3pm == 0).astype(int).sum(axis=0)

88

In [22]:
# Imputing Hillshade_3pm zeros with the median
simp = SimpleImputer(missing_values=0, strategy='median')
# fit_transform requires X as a numpy array of shape [n_samples, n_features]
# thus the dataframe column is casted to a numpy array and reshaped
train.Hillshade_3pm = simp.fit_transform(train.Hillshade_3pm.values.reshape(-1,1))
# The imputer upcast the df column to float64, we don't need that
train.Hillshade_3pm = train.Hillshade_3pm.astype(np.float32)
# Count the zeros again to check the result
(train.Hillshade_3pm == 0).astype(int).sum(axis=0)

0

In [23]:
# Drop the training set from df_covtype
df_covtype = df_covtype.iloc[15120:]
# Append the imputed training set to df_covtype
df_covtype = pd.concat([train, df_covtype])

In [24]:
# Don't need train df anymore
del train

In [25]:
# Let's compute the Euclidean distance to Hydrology
hv_distances_labels = ['Horizontal_Distance_To_Hydrology',
                       'Vertical_Distance_To_Hydrology']
hv_dist_hydro_arr = df_covtype[hv_distances_labels].values
# Perform the Euclidean distance with respect to the origin
# (i.e (0;0) where the water is located)
euc_distance_to_hydro = np.linalg.norm(hv_dist_hydro_arr, axis=1)
# Add the new feature 'Distance_To_Hydrology' to the training set
# just after the 'Slope' feature at index 2
# rounding each distance to four decimal places
df_covtype.insert(3,
                  'Distance_To_Hydrology',
                  np.around(euc_distance_to_hydro, decimals=4))
# Drop the horizontal and vertical distance
df_covtype.drop(columns=hv_distances_labels, inplace=True)

In [26]:
# Check the result
df_covtype.head(2)

,Elevation,Aspect,Slope,Distance_To_Hydrology,Horizontal_Distance_To_Roadways,Hillshade_9am,Hillshade_Noon,Hillshade_3pm,Horizontal_Distance_To_Fire_Points,Wilderness_Area_1,...,Soil_Type_32,Soil_Type_33,Soil_Type_34,Soil_Type_35,Soil_Type_36,Soil_Type_37,Soil_Type_38,Soil_Type_39,Soil_Type_40,Cover_Type
0,2596,51,3,258.0000,510,221,232,148.0,6279,1,...,0,0,0,0,0,0,0,0,0,5
1,2590,56,2,212.0849,390,220,235,151.0,6225,1,...,0,0,0,0,0,0,0,0,0,5


In [27]:
# Perform one hot encoding via get_dummies,
# then drop the integer target label, leaving the one hot encoded labels only
one_hot_covtype = pd.get_dummies(df_covtype.Cover_Type, prefix='CovT')
df_covtype.drop(columns='Cover_Type', inplace=True)
df_covtype_ohe = df_covtype.join(one_hot_covtype)

In [28]:
# Check the resulting shape
df_covtype_ohe.shape

(581012, 60)

In [29]:
# Don't need df_covtype anymore
print('Dereferencing df_covtype')
del df_covtype

Dereferencing df_covtype


In [30]:
# Drop Aspect and Slope since working with angles is tricky
# when computing the mean
df_covtype_ohe.drop(labels=['Aspect', 'Slope'], axis=1, inplace=True)

In [31]:
def train_test_split(df_covtype_ohe):
    '''
    Split the one hot encoded dataset onto training set and test set
    according to UCI's repository guidelines
    '''
    # First 15120 rows for the training set
    X_train = df_covtype_ohe[:15120].copy()
    # The last seven colums are the targets
    X_train, y_train = X_train.iloc[:, :51], X_train.iloc[:, 51:]
    # The remaining rows are for the test set
    X_test = df_covtype_ohe[15121:].copy()
    X_test, y_test = X_test.iloc[:, :51], X_test.iloc[:, 51:]
    return X_train, X_test, y_train, y_test

X_train, X_test, y_train, y_test = train_test_split(df_covtype_ohe)

In [32]:
# Check shapes consistency
print(f'X_train: {X_train.shape}, X_test: {X_test.shape}, ' \
      f'y_train: {y_train.shape}, y_test: {y_test.shape}')

X_train: (15120, 51), X_test: (565891, 51), y_train: (15120, 7), y_test: (565891, 7)


In [33]:
# Let's standardize the training set and test set.
# Note that we use the training set ONLY to calculate the mean and standard deviation
# then normalize the training set 
# and finally use the (training) mean and standard deviation to normalize the test set.
# This ensures no data leakage.

def train_test_normalize(X_train, X_test):
    '''
    Perform standardization on the training set and transforms the
    test set accordingly
    '''
    # The numerical columns we want to normalize
    numerical_columns = ['Elevation',
                         'Distance_To_Hydrology',
                         'Horizontal_Distance_To_Roadways',
                         'Hillshade_9am', 'Hillshade_Noon', 'Hillshade_3pm',
                         'Horizontal_Distance_To_Fire_Points']
    # Calculate the mean and standard deviation of the training set
    X_train_num_cols_mean = X_train[numerical_columns].mean()
    X_train_num_cols_std = X_train[numerical_columns].std()
    # Perform standardization over the numerical columns of the training set
    X_train_std = (X_train[numerical_columns] - X_train_num_cols_mean) / X_train_num_cols_std
    # Concatenate side-by-side the normalized training set and the one-hot encoded features
    # Note that we index X_train dataframe by the (set) difference of the overall features
    # minus the numerical ones
    ohe_features = X_train.columns.difference(other=numerical_columns, sort=False)
    X_train_std = pd.concat([X_train_std, X_train[ohe_features]], axis=1)
    # Perform standardization over the numerical columns of the test set, using the mean and std
    # of the training set as discussed earlier
    X_test_std = (X_test[numerical_columns] - X_train_num_cols_mean) / X_train_num_cols_std
    # Concatenate side-by-side the normalized test set and the one-hot encoded features
    X_test_std = pd.concat([X_test_std, X_test[ohe_features]], axis=1)
    return X_train_std, X_test_std

X_train_std, X_test_std = train_test_normalize(X_train, X_test)

In [34]:
# Initialize the training/validation set splitter.
# We'll use it later for the grid-search, performing a 10-fold cross validation
# where each validation set is 25% the size of the training set.
# This yields 1620 samples for the new training set and 540 for each validation set.
validation_strat = StratifiedShuffleSplit(n_splits=10, test_size=0.25, random_state=0)

In [36]:
# wrapper function which builds the nn classifier
def make_model(n_features=51, n_classes=7, dense_layer_size=120, learning_rate=0.5):
    model = keras.Sequential()
    model.add(keras.layers.Dense(dense_layer_size, activation='relu', input_dim=n_features))
    model.add(keras.layers.Dense(n_classes, activation='softmax'))
    sgd = keras.optimizers.SGD(lr=learning_rate) # per il momentum: momentum=0.9
    # TODO: CONTROLLARE MEGLIO L'INTERAZIONE TRA categorical_crossentropy E accuracy
    # SENZA CROSS VALIDATION AVEVO UTILIZZATO categorical_accuracy PERCHE' SEMBRA ESSERE
    # MAGGIORMENTE INDICATA!!
    # IL MISTERO SI INFITTISCE. SE VOLESSIMO USARE F1-SCORE, PREC-RECALL, COPIARE
    # IL CODICE DI unnir da qui
    # https://github.com/keras-team/keras/issues/5400#issuecomment-314747992
    model.compile(optimizer=sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    model.summary()
    return model

# Wrap the Keras neural network onto a scikit Classifier
classifier_net = keras.wrappers.scikit_learn.KerasClassifier(make_model)

# For params tuning see
# https://machinelearningmastery.com/grid-search-hyperparameters-deep-learning-models-python-keras/

# Define the grid search params
# uncomment for changing the default number of features and/or classes
# for the network. DON'T FORGET TO CHANGE THE TRAINING/TEST SET ACCORDINGLY!
# n_features = [X_train_std.shape[1]]
# n_classes = [y_train.shape[1]]
dense_layer_size = [120]
learning_rate = [0.001, 0.01, 0.1, 0.5]
batch_size = [1134]
epochs = [700]
# Define the grid of parameters 
param_grid = dict(# n_features=n_features, # uncomment if needed
                  # n_classes=n_classes,   # uncomment if needed
                  dense_layer_size=dense_layer_size,
                  learning_rate=learning_rate,
                  batch_size=batch_size,
                  epochs=epochs)
# Initialize the grid search using the nn classifier and the cross-validation
# strategy defined above. Change n_jobs to 1 if running on GPU, -1 is fine for CPU runs.
# PER OTTENERE I VALORI y_pred per ogni iterazione della cross-val leggere qui
# https://stackoverflow.com/questions/49633465/gridsearchcv-access-to-predicted-values-across-tests
# per classification_report leggere qui
# https://stackoverflow.com/questions/42562146/classification-report-with-nested-cross-validation-in-sklearn
grid = GridSearchCV(estimator=classifier_net,
                    param_grid=param_grid,
                    n_jobs=-1,
                    cv=validation_strat)

W0629 17:53:40.761591 139794643023680 deprecation.py:506] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/ops/init_ops.py:1251: calling VarianceScaling.__init__ (from tensorflow.python.ops.init_ops) with dtype is deprecated and will be removed in a future version.
Instructions for updating:
Call initializer instance with the dtype argument instead of passing it to the constructor


Epoch 1/700
11340/11340 [==============================] - 0s 20us/sample - loss: 1.9530 - acc: 0.1817
Epoch 2/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.9499 - acc: 0.1838
Epoch 3/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.9468 - acc: 0.1853
Epoch 4/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.9438 - acc: 0.1872
Epoch 5/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.9407 - acc: 0.1890
Epoch 6/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.9377 - acc: 0.1912
Epoch 7/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.9347 - acc: 0.1938
Epoch 8/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.9318 - acc: 0.1958
Epoch 9/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.9288 - acc: 0.1975
Epoch 10/700
11340/11340 [==============================] - 0s 2us/sample - loss:

11340/11340 [==============================] - 0s 2us/sample - loss: 1.6172 - acc: 0.4381
Epoch 160/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.6157 - acc: 0.4384
Epoch 161/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.6142 - acc: 0.4397
Epoch 162/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.6126 - acc: 0.4407
Epoch 163/700
11340/11340 [==============================] - 0s 3us/sample - loss: 1.6111 - acc: 0.4412
Epoch 164/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.6096 - acc: 0.4426
Epoch 165/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.6081 - acc: 0.4437
Epoch 166/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.6066 - acc: 0.4445
Epoch 167/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.6051 - acc: 0.4450
Epoch 168/700
11340/11340 [==============================] - 0s 2us/sample - l

11340/11340 [==============================] - 0s 2us/sample - loss: 1.4268 - acc: 0.5116
Epoch 316/700
11340/11340 [==============================] - 0s 3us/sample - loss: 1.4258 - acc: 0.5118
Epoch 317/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.4249 - acc: 0.5123
Epoch 318/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.4239 - acc: 0.5126
Epoch 319/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.4229 - acc: 0.5129
Epoch 320/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.4220 - acc: 0.5131
Epoch 321/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.4210 - acc: 0.5137
Epoch 322/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.4201 - acc: 0.5140
Epoch 323/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.4191 - acc: 0.5143
Epoch 324/700
11340/11340 [==============================] - 0s 2us/sample - l

11340/11340 [==============================] - 0s 2us/sample - loss: 1.3007 - acc: 0.5414
Epoch 472/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.3000 - acc: 0.5416
Epoch 473/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.2994 - acc: 0.5421
Epoch 474/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.2987 - acc: 0.5422
Epoch 475/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.2980 - acc: 0.5425
Epoch 476/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.2973 - acc: 0.5425
Epoch 477/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.2967 - acc: 0.5426
Epoch 478/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.2960 - acc: 0.5426
Epoch 479/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.2954 - acc: 0.5427
Epoch 480/700
11340/11340 [==============================] - 0s 2us/sample - l

...

11340/11340 [==============================] - 0s 2us/sample - loss: 0.8018 - acc: 0.6888
Epoch 293/700
11340/11340 [==============================] - 0s 3us/sample - loss: 0.8012 - acc: 0.6884
Epoch 294/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.8006 - acc: 0.6891
Epoch 295/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.8001 - acc: 0.6897
Epoch 296/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7995 - acc: 0.6897
Epoch 297/700
11340/11340 [==============================] - 0s 3us/sample - loss: 0.7989 - acc: 0.6899
Epoch 298/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7984 - acc: 0.6899
Epoch 299/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7978 - acc: 0.6907
Epoch 300/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7972 - acc: 0.6907
Epoch 301/700
11340/11340 [==============================] - 0s 3us/sample - l

11340/11340 [==============================] - 0s 2us/sample - loss: 0.7396 - acc: 0.7056
Epoch 449/700
11340/11340 [==============================] - 0s 3us/sample - loss: 0.7394 - acc: 0.7059
Epoch 450/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7391 - acc: 0.7060
Epoch 451/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7388 - acc: 0.7055
Epoch 452/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7386 - acc: 0.7061
Epoch 453/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7383 - acc: 0.7058
Epoch 454/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7380 - acc: 0.7062
Epoch 455/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7377 - acc: 0.7063
Epoch 456/700
11340/11340 [==============================] - 0s 3us/sample - loss: 0.7374 - acc: 0.7064
Epoch 457/700
11340/11340 [==============================] - 0s 3us/sample - l

11340/11340 [==============================] - 0s 3us/sample - loss: 0.7053 - acc: 0.7180
Epoch 605/700
11340/11340 [==============================] - 0s 3us/sample - loss: 0.7052 - acc: 0.7178
Epoch 606/700
11340/11340 [==============================] - 0s 3us/sample - loss: 0.7050 - acc: 0.7175
Epoch 607/700
11340/11340 [==============================] - 0s 3us/sample - loss: 0.7048 - acc: 0.7179
Epoch 608/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7046 - acc: 0.7178
Epoch 609/700
11340/11340 [==============================] - 0s 3us/sample - loss: 0.7045 - acc: 0.7180
Epoch 610/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7043 - acc: 0.7182
Epoch 611/700
11340/11340 [==============================] - 0s 2us/sample - loss: 0.7041 - acc: 0.7178
Epoch 612/700
11340/11340 [==============================] - 0s 3us/sample - loss: 0.7039 - acc: 0.7184
Epoch 613/700
11340/11340 [==============================] - 0s 2us/sample - l

11340/11340 [==============================] - 0s 3us/sample - loss: 1.2135 - acc: 0.5698
Epoch 61/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.2082 - acc: 0.5708
Epoch 62/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.2031 - acc: 0.5720
Epoch 63/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.1980 - acc: 0.5733
Epoch 64/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.1931 - acc: 0.5745
Epoch 65/700
11340/11340 [==============================] - 0s 3us/sample - loss: 1.1882 - acc: 0.5768
Epoch 66/700
11340/11340 [==============================] - 0s 3us/sample - loss: 1.1835 - acc: 0.5777
Epoch 67/700
11340/11340 [==============================] - 0s 3us/sample - loss: 1.1788 - acc: 0.5787
Epoch 68/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.1741 - acc: 0.5791
Epoch 69/700
11340/11340 [==============================] - 0s 2us/sample - loss: 1.16

KeyboardInterrupt: 

In [ ]:
# Run grid search
# THIS CELL IS TIME CONSUMING, BE AWARE OF THAT!
grid.fit(X_train_std, y_train)

In [ ]:
# Print the best scores defined earlier for the grid search and its model params
print(f'Best scores: {grid.best_score_:.4f} using {grid.best_params_}', end='\n\n')
# Get the mean of each score for each cross-validation run
means = grid.cv_results_['mean_test_score']
# Get the standard dev of each score for each cross-validation run
stds = grid.cv_results_['std_test_score']
# Get the model params for each cross-validation run
params = grid.cv_results_['params']
# Loop through means, stds, params and print
# one line for each cross-validation run
for mean, stdev, param in zip(means, stds, params):
    print(f'Mean: {mean:.4f} +- {stdev:.6f} with: {param}')

from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(C=0.1, random_state=1, solver='liblinear', multi_class='ovr')
lr.fit(X_train_std, y_train)

print('Logistic Regression, training accuracy: %.2f%%' % (100 * lr.score(X_train_std, y_train)))